## Задание № 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic;
- для источников paid и email из России - ставим ad;
- для источников paid и email не из России - ставим other;
- все остальные варианты берем из traffic_source без изменений.

In [81]:
import pandas as pd
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [82]:
import pandas as pd
log.loc[log['traffic_source'].str.contains('yandex|google'), 'traffic_type'] = 'organic'

log_rus = log.loc[log.region == 'Russia']
log_rus.loc[log_rus['traffic_source'].str.contains('email|paid'), 'traffic_type'] = 'ad'

log_notrus = log.loc[log.region != 'Russia']
log_notrus.loc[log_notrus['traffic_source'].str.contains('email|paid'), 'traffic_type'] = 'other'

log = log_rus.merge(log_notrus, how='outer')
log.loc[pd.isnull(log.traffic_type), 'traffic_type'] = log['traffic_source']

log

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094245,1e0d086733,https://host.ru/c2382eb3d6afc8d0f3,Ukraine,10c296a6a4,paid,other
18934,1550094255,c7e4bce62a,https://host.ru/8880fd00cf23066541,Australia,e721d23ec6,yandex,organic
18935,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,direct
18936,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other


## Задание № 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

- прочитайте содержимое файла с датафрейм
- отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [50]:
import pandas as pd
URL = pd.read_csv('URLs.txt', sep=';')
URL.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [54]:
import pandas as pd
URL[URL['url'].str.contains('/\d{8}-')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


## Задание № 3

В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [21]:
import pandas as pd
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

from pymystem3 import Mystem
def add_lemma(row):  
    list_lemmas = Mystem().lemmatize(row['keyword'])
    lemmas = ''.join(list_lemmas)
    lemma = lemmas.replace('\n', '')
    return lemma
    
data['lemmas'] = data.apply(add_lemma, axis=1)
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль
1,доллары в рубли,114173,доллар в рубль
2,100 долларов в рублях,97534,100 доллар в рубль
3,курс рубля,53546,курс рубль
